In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os.path
import psycopg2

import configparser

In [2]:
# Path to the sql script used for getting images and details from database.
SELECT_IMAGE_SCRIPT_PATH = "database/select_aircraft_images.sql"

# Used as a performance argument.
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Batch size for data set to process.
BATCH_SIZE = 32

# The file used for caching the dataset.
DATASET_CACHE_FILE = "dataset_cache"

IMG_HEIGHT = 128
IMG_WIDTH = 128

In [3]:
class DBConfig:
    def __init__(self, filename, host, port, db_name, user, pwd):
        self.filename = filename
        self.host = host
        self.port = port
        self.db_name = db_name
        self.user = user
        self.pwd = pwd

In [4]:
def load_config(filename = 'database.ini', section = 'aircraft_postgres_db'):
    parser = configparser.ConfigParser()
    parser.read(filename)
    cfg = parser[section]
    return DBConfig(filename, cfg['Host'], int(cfg['Port']), cfg['Db_name'], cfg['User'], cfg['Pass'])

In [5]:
def load_sql(filename):
    with open(filename, "r") as sql_script_file:
        return sql_script_file.read()

In [66]:
# This is obviously not a good solution but hopefully it will work temporarly as a learning opportunity.
VARIANTS = ['737-800', 'c-130', 'cessna 172']

def convert_to_number(variant_name):
    string = tf.strings.unicode_encode(variant_name, 'UTF-8')
    print(string)
#     return VARIANTS.index(string)
    return 0

In [67]:
def ready_image(image_details):
    # Path must be transformed into the actual image.
    path = image_details[0]
    
    # Variant name is a string which cannot be used as a target - must be mapped to a number.
    variant_name = image_details[1]
    
    variant_name = convert_to_number(variant_name)
    
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    return img, variant_name

In [68]:
def prepare_image_dataset(image_paths_dataset):
    ds = image_paths_dataset.map(ready_image, num_parallel_calls=AUTOTUNE)
    
    ds = ds.cache(DATASET_CACHE_FILE)
    
    
    # ds = ds.repeat()
    
    
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds
    

In [69]:
def train_on_images(image_paths):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths);
    for (path, variant_name) in image_paths:
        img = read_parse_image(path)

In [70]:
# Taken from https://www.tensorflow.org/tutorials/load_data/images (09/06/2020)
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(label_batch[n].title())
      plt.axis('off')

In [71]:
db_cfg = load_config()

In [72]:
conn = psycopg2.connect(dbname=db_cfg.db_name, user=db_cfg.user, password=db_cfg.pwd, host=db_cfg.host, port=db_cfg.port)

In [73]:
cur = conn.cursor()
cur.execute('SELECT version()')
print("Connected to database, version: " + str(cur.fetchone()))

Connected to database, version: ('PostgreSQL 12.3, compiled by Visual C++ build 1914, 64-bit',)


In [74]:
select_image_sql = load_sql(SELECT_IMAGE_SCRIPT_PATH)

In [75]:
cur.execute(select_image_sql)

In [76]:
image_paths = cur.fetchall()

image_count = len(image_paths)

In [77]:
image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)

image_dataset = prepare_image_dataset(image_dataset)

img_batch, label_batch = next(iter(image_dataset))

show_batch(img_batch.numpy(), label_batch.numpy())


ValueError: in user code:

    <ipython-input-24-8beedc421df6>:8 ready_image  *
        variant_name = convert_to_number(variant_name)
    <ipython-input-66-af403b75bdc4>:5 convert_to_number  *
        string = tf.strings.unicode_encode(variant_name, 'UTF-8')
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\ops\ragged\ragged_string_ops.py:163 unicode_encode  **
        raise ValueError("input_tensor's rank must be at least 1.")

    ValueError: input_tensor's rank must be at least 1.


# Data is now loaded and ready, time to actually handle the model

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [50]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [51]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)            

In [53]:
batch_size = 128
epochs = 15

history = model.fit(image_dataset)

ValueError: in user code:

    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    C:\Users\paul\anaconda3\envs\venv_air\lib\site-packages\tensorflow\p